In [1]:
import os
os.environ['R_HOME'] = '/Users/alexe/anaconda3/envs/r/lib/R'
os.environ['PATH'] = f"{os.environ['R_HOME']}/bin:" + os.environ['PATH']

import numpy as np
from rpy2.robjects import numpy2ri
import rpy2.robjects as ro


In [2]:
# Load and convert data as before
pre_raw_data = np.load('/Users/alexe/Kaggle/Till_Bertil/pre_raw_data.npy')
post_raw_data = np.load('/Users/alexe/Kaggle/Till_Bertil/post_raw_data.npy')
evoked_pre_data = np.load('/Users/alexe/Kaggle/Till_Bertil/evoked_pre_data.npy')
evoked_post_data = np.load('/Users/alexe/Kaggle/Till_Bertil/evoked_post_data.npy')

# Activate automatic conversion
numpy2ri.activate()

# Convert and assign to R's global environment
ro.globalenv['r_pre_raw'] = numpy2ri.py2rpy(pre_raw_data)
ro.globalenv['r_post_raw'] = numpy2ri.py2rpy(post_raw_data)
ro.globalenv['r_evoked_pre'] = numpy2ri.py2rpy(evoked_pre_data)
ro.globalenv['r_evoked_post'] = numpy2ri.py2rpy(evoked_post_data)

# Save to RDS files
ro.r('saveRDS(r_pre_raw, file="/Users/alexe/Kaggle/Till_Bertil/pre_raw.rds")')
ro.r('saveRDS(r_post_raw, file="/Users/alexe/Kaggle/Till_Bertil/post_raw.rds")')
ro.r('saveRDS(r_evoked_pre, file="/Users/alexe/Kaggle/Till_Bertil/evoked_pre.rds")')
ro.r('saveRDS(r_evoked_post, file="/Users/alexe/Kaggle/Till_Bertil/evoked_post.rds")')


<rpy2.rinterface_lib.sexp.NULLType object at 0x111955080> [RTYPES.NILSXP]

In [ ]:
import os
import numpy as np
import mne

def process_epo_files(input_dir, output_dir):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Find all -epo.fif files in the input directory
    epo_files = [f for f in os.listdir(input_dir) if f.endswith('-epo.fif')]

    # Process each file
    for epo_file in epo_files:
        try:
            # Load epochs
            epochs = mne.read_epochs(os.path.join(input_dir, epo_file))

            # Compute average and get data
            evoked_data = epochs.average().get_data()

            # Create output filename (replace -epo.fif with .npy)
            output_file = epo_file.replace('-epo.fif', '.npy')

            # Save as numpy array
            np.save(os.path.join(output_dir, output_file), evoked_data)

            print(f"Processed {epo_file} successfully")

        except Exception as e:
            print(f"Error processing {epo_file}: {str(e)}")

# Usage example:
input_directory = '/Users/alexe/Kaggle/get_done/TMSEEG/out'
output_directory = '/Users/alexe/Kaggle/Till_Bertil/Evoked_matriser_med_ICA_Magnus'
process_epo_files(input_directory, output_directory)


In [5]:
import re

def clean_filename(filename):
    # Remove any characters that might cause issues
    clean = re.sub(r'[^a-zA-Z0-9_]', '_', filename)
    return clean

def convert_npy_to_rds(input_dir, output_dir=None):
    # If no output directory specified, use input directory
    if output_dir is None:
        output_dir = input_dir

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Activate automatic conversion
    numpy2ri.activate()

    # Find all .npy files in the input directory
    npy_files = [f for f in os.listdir(input_dir) if f.endswith('.npy')]

    for npy_file in npy_files:
        try:
            # Load numpy array
            data = np.load(os.path.join(input_dir, npy_file))

            # Clean the filename for R object name
            clean_name = clean_filename(os.path.splitext(npy_file)[0])
            r_name = 'r_' + clean_name

            # Convert and assign to R's global environment
            ro.globalenv[r_name] = numpy2ri.py2rpy(data)

            # Create output filename (keep original filename for the .rds file)
            rds_file = os.path.join(output_dir, npy_file.replace('.npy', '.rds'))

            # Use cleaned name in R command but original name for file
            r_command = f'saveRDS({r_name}, file="{rds_file}")'
            print(f"Executing R command: {r_command}")  # Debug print
            ro.r(r_command)

            print(f"Converted {npy_file} to {os.path.basename(rds_file)}")

        except Exception as e:
            print(f"Error processing {npy_file}: {str(e)}")
            print(f"Data shape: {data.shape}")  # Print shape for debugging
            print(f"Data type: {data.dtype}")   # Print type for debugging

# Usage example:
input_directory = '/Users/alexe/Kaggle/Till_Bertil/Evoked_matriser_med_ICA_Magnus'
convert_npy_to_rds(input_directory)

Executing R command: saveRDS(r_2024_05_31T143456_phase_4_preproc, file="/Users/alexe/Kaggle/Till_Bertil/Evoked_matriser_med_ICA_Magnus/2024-05-31T143456_phase_4_preproc.rds")
Converted 2024-05-31T143456_phase_4_preproc.npy to 2024-05-31T143456_phase_4_preproc.rds
Executing R command: saveRDS(r_2023_12_19T152518_phase_2_preproc, file="/Users/alexe/Kaggle/Till_Bertil/Evoked_matriser_med_ICA_Magnus/2023-12-19T152518_phase_2_preproc.rds")
Converted 2023-12-19T152518_phase_2_preproc.npy to 2023-12-19T152518_phase_2_preproc.rds
Executing R command: saveRDS(r_2023_12_19T143454_phase_1_preproc, file="/Users/alexe/Kaggle/Till_Bertil/Evoked_matriser_med_ICA_Magnus/2023-12-19T143454_phase_1_preproc.rds")
Converted 2023-12-19T143454_phase_1_preproc.npy to 2023-12-19T143454_phase_1_preproc.rds
Executing R command: saveRDS(r_2024_12_04T170714_sleep_phase_1_preproc, file="/Users/alexe/Kaggle/Till_Bertil/Evoked_matriser_med_ICA_Magnus/2024-12-04T170714_sleep_phase_1_preproc.rds")
Converted 2024-12-04T